First, import libraries

In [18]:
import folium
import requests
import pandas as pd

Madrid is a citi that receives a lot of tourism. In order to analyze if it's interesting to set a business oriented to tourism, we need to know if the location is really important or not. To do so, I will check if the main venues are in the center of the city.

I start by loading the Madrid map

In [60]:
latitude = 40.4168
longitude = -3.7038

map_madrid = folium.Map(location=[latitude, longitude], zoom_start=15)

In [61]:
map_madrid

After that, I set my credentials to run the Foursquare query:

In [63]:
CLIENT_ID = 'VQUC23XR54K05DSOIIRFUJTN0SDL1REBPONIAM1LDITOUIRI' # your Foursquare ID
CLIENT_SECRET = '5IDPFEMDTLDTSTBXDPF105TW2EPYKHWR4VU3J0D0C1VUEU5Y' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VQUC23XR54K05DSOIIRFUJTN0SDL1REBPONIAM1LDITOUIRI
CLIENT_SECRET:5IDPFEMDTLDTSTBXDPF105TW2EPYKHWR4VU3J0D0C1VUEU5Y


In [64]:
radius = 10000
limit = 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,VERSION,latitude,longitude, radius, limit)

I get the results of the query. Now I should convert the important information (name and location) to a pandas dataframe.

In [65]:
results = requests.get(url).json()
results = results['response']['venues']

In [66]:
dicc = {}
dicc['id'] = []
dicc['name'] = []
dicc['lat'] = []
dicc['lng'] = []
for result in results:
        dicc['name'].append(result['name'])
        dicc['lat'].append(result['location']['lat'])
        dicc['lng'].append(result['location']['lng'])

In [67]:
df = pd.DataFrame()
df['name'] = dicc['name']
df['lat'] = dicc['lat']
df['lng'] = dicc['lng']
df.head()

,name,lat,lng
0,Puerta del Sol,40.417027,-3.703443
1,Kilómetro 0,40.416831,-3.703840
2,Metro Sol,40.416882,-3.703038
3,Chocolatería San Ginés,40.416754,-3.707079
4,Plaza Mayor,40.415527,-3.707506


Now, let's add the markers of the venues to de Madrid map

In [68]:
for lat, lng, name in zip(df['lat'], df['lng'], df['name']):
    label = name
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid) 

In [69]:
map_madrid

We can check that almost all of the venues are located in the center, so location is very important.